# The structures notebook  <a id="summary-truncation"></a>[<font size=1>(back to `Main.ipynb`)</font>](./Main.ipynb)

This notebook gathers all structures -- and their related constructors -- that are used in the computaion of the steady-state Ramsey allocation.

# Input

## Structure

In [1]:
struct Economy{T<:Real,T1<:Function,T2<:Function,T3<:Function,T4<:Function,T5<:Function,T6<:Function,T7<:Function,T8<:Function,I<:Int64}
    β::T      # discount factor
    α::T      # capital share in the Cobb-Douglas production function
    δ::T      # capital depreciation
    γ::T      # inverse of consumption IES (needed for Dynare only)
    χ::T      # scaling labor supply
    ϕ::T      # Frish elasticity
    τl::T     # labor tax
    τk::T     # capital tax
    τs::T     # labor subsidy
    Tt::T     # transfers

    #=
        *** wage and price rigidities ***
    =#
    ψp::T     # price adj cost
    ψw::T     # wage adj cost
    ϵp::T     # market power firm
    ϵw::T     # market power union
    κw::T     # corrective factor for the steady-state labor wedge
    
    #=
        *** utility functions ***
    =#
    u  ::T1        # utility function for consumption
    u′ ::T2       # u′
    inv_u′::T3       # inverse of u′
    u′′::T4     # second-order derivative of u 
    v  ::T5     # second-order derivative of u 
    v′ ::T6     # second-order derivative of u 
    v′′::T7     # second-order derivative of u
    inv_v′::T8
    
    #=
        *** grid for asset choices ***
    =#
    na    ::I         # number of grid points
    a_min ::T         # min asset holdings
    aGrid ::Vector{T} # grid for asset choices

    #=
        *** productivity process ***
    =#
    ny    ::I         # number of productivity states
    ys    ::Vector{T} # productivity states  
    Πy    ::Matrix{T} # transition matrix 
    Sy    ::Vector{T} # productivity distribution 
end;


## Constructor

In [1]:
function Economy(;
    α::T      = 0.36,
    # β::T      = 0.99, #discount factor
    β::T      = 9891196834817012,
    δ::T      = 0.025,
    ϕ::T      = 0.5,
    γ::T      = 2.0,
    τk::T     = 0.36,
    # χ::T      = 1.0,       
    χ::T      = 0.033,
    τl::T     = 0.28, # initial guess
    τs::T     = 0.0, # be careful, it is 1-τ in Heathcote and κ is λ ! 
    Tt::T     = 0.114,
    ψp::T     = 0.0,  #price adj cost
    ψw::T     = 100.0, #wage adj cost
    ϵp::T     = 6.0,  #market power firm
    ϵw::T     = 21.0, #market power union
    κw::T     = 1.0,
    ρy        = 0.994,            # quarterly persistence of productivity
    σy        = 0.0727,              # quarterly SE of productivity innovation
    # ρy::T     = 0.99,      # version sticky price
    # σy::T     = 0.123,      # version sticky price
    ny::I     = 5,
    na::I     = 100,
    a_min::T  = 1e-9,
    a_max::T  = 1000.0, #10000
    curv_a::I = 4
    ) where {T<:Real,I<:Int}
    
    Rt = one(T)/β #before-tax gross interest rate
    wt = (one(T)-α)*( (Rt - (1-δ))/α )^(α/(α-one(T)))
    
    R  = (one(T) + (one(T) - τk)*(Rt-one(T)))

    
    # κw= 1/(1-1/ϵw) # κw is 1/(1-tauE)
    # tauE = 1 - 1/κw
    # w  = (1-τl)*(1-τs)*(1-tauE)*one(T) # their tax allowed wt = 1? 
    w  = (1-τl)*wt #  


    # w  = (1-τl)*(1-τs)*(1-tauE)*wt 


    
    # χ  = ((1/3.508)^(1/ϕ))/w 
    #χ  = 0.01# to targer L = 0.33 0.12


    #κw = 1/(1-1/ϵw) # κw is 1/(1-tauE)


    #=
        *** Constructing utility functions ***
    =#
    u(c)       = (γ ≈ one(T)) ? log(c) : (c^(one(T)-γ)-zero(T))/(one(T)-γ)::T
    u′(c)      = c^(-γ)::T
    inv_u′(up) = up^(-one(T)/γ)::T 
    u′′(c)     = -γ*c^(-γ-one(T))::T
    v(l)       = (1/χ)*(l^(1+1/ϕ))/(1+1/ϕ) # function v
    v′(l)      = (1/χ)*l^(1/ϕ)
    inv_v′(vp) = (χ*vp)^ϕ
    v′′(l)     = (1/χ)*(1/ϕ)*l^(1/ϕ-1)    

    #=
        *** Constructing the asset grid ***
    =#    
    aGrid = a_min .+ (a_max-a_min)*(range(0.0, 1.0, length=na)).^curv_a
    
    #=
        *** Constructing the productivity process ***
    =#
    mc    =  rouwenhorst(ny,ρy,σy)
    Trans = collect(mc.p')
    Trans[findall(x->x<=5*10^-5,Trans)] .= zero(Trans[1,1])
    for i = 1:ny
        Trans[i,i] += one(Trans[1,1]) - sum(Trans,dims=1)[i]
    end
    Sy  = (Trans^100000)[:,1]
    
    endow = exp.(mc.state_values)
    ys = endow./sum(Sy.*endow) # ensuring L=1
    
     #return the Params using the natural constructor 
    return Economy{T,typeof(u),typeof(u′),typeof(inv_u′),
                    typeof(u′′),typeof(v),typeof(v′),typeof(v′′),typeof(inv_v′),I}(
        β,α,δ,γ,χ,ϕ,τl,τk,τs,Tt,
        ψp,ψw,ϵp,ϵw,κw,
        u,u′,inv_u′,u′′,v,v′,v′′,inv_v′,
        na,a_min,aGrid,
        ny,ys,Matrix(Trans'),Sy)
        
end;

function Economy(slope_p::T,# slope Philipps curve pricve
    slope_w::T,# slope Phillips curve wage
    eco::Economy) where {T<:Real}
    #changing only the price adjustment cost

    Economy(eco.β,eco.α,eco.δ,eco.γ,eco.χ,eco.ϕ,eco.τl,eco.τk,eco.τs,eco.Tt,
        (eco.ϵp-1)/slope_p,eco.ϵw/slope_w,eco.ϵp,eco.ϵw,eco.κw,
        eco.u,eco.u′,eco.inv_u′,eco.u′′,eco.v,eco.v′,eco.v′′,eco.inv_v′,
        eco.na,eco.a_min,eco.aGrid,
        eco.ny,eco.ys,eco.Πy,eco.Sy)
end;

function Economy(eco::Economy; 
    slope_p::T=0.05,# slope Philipps curve pricve
    slope_w::T=0.05# slope Phillips curve wage
    ) where {T<:Real}
    
    Economy(slope_p,slope_w,eco)        
end;

LoadError: TypeError: in Type{...} expression, expected UnionAll, got a value of type typeof(Economy)

# Output of the Aiyagari model

## Structure 

In [3]:
mutable struct AiyagariSolution{T<:Real,I<:Integer}    
    ga::Matrix{T} # policy function for savings on the asset grid
    gl::Matrix{T} # policy function for labor supply on the asset grid
    gc::Matrix{T} # policy function for consumption on the asset grid
    Rt::T         # pre-tax gross interest rate
    wt::T         # pre-tax wage rate
    R::T          # post-tax gross interest rate
    w::T          # post-tax wage rate
    A::T          # aggregate savings
    C::T          # aggregate consumption
    K::T          # aggregate capital
    L::T          # aggregate and individual labor supply (in efficient units)
    G::T          # public spending
    Y::T          # GDP
    B::T          # public debt
#     πw::T         # wage inflation
#     πp::T         # price inflation
    transitMat::SparseMatrixCSC{T,I} #
    stationaryDist::Matrix{T} #
    residEuler::Matrix{T} #
end

## Constructor

In [4]:
function AiyagariSolution(eco::Economy{T,T1,T2,T3,T4,T5,T6,T7,T8,I}) where {T<:Real,
            T1<:Function,T2<:Function,T3<:Function,T4<:Function,T5<:Function,
            T6<:Function,T7<:Function,T8<:Function,I<:Int64}
    @unpack β,α,δ,γ,τs,τk,τl,ys,na,u′,ny,aGrid = eco
    Rt = one(T)/β #before-tax gross interest rate
    # wt = one(T)  #(one(T)-α)*( (Rt - (1-δ))/α )^(α/(α-one(T)))
    wt = (one(T)-α)*( (Rt - (1-δ))/α )^(α/(α-one(T)))
    # R = (one(T) + (one(T) - τk)*(1.0025-one(T)))
    R  = (one(T) + (one(T) - τk)*(Rt-one(T)))

#     w = (1-τl)*(1-τs)*one(T)
    w = (1-τl)*wt
    L = one(T)/3one(T)
    cs = repeat(ys', outer=(na,1))
    
    ls = fill(L,size(cs))
    
    as = repeat(aGrid, outer=(1,ny))
    @assert size(cs) == size(as) == (na,ny)
    
    A  = sum(as)
    return AiyagariSolution{T,I}(as,ls,cs,Rt,wt,R,w,A,A,A,L,
                                    zero(T),A*L,zero(T),
                                    spzeros(T, I, na*ny,na*ny), 
                                    fill(one(T)/(na*ny), na,ny), zeros(T,na,ny))
end

# Truncated model

## Structures

We consider three structures for the truncated model:

* `TruncatedAllocation` containing the allocations at the truncated history level (as well as sizes and transition matrix);
* `ξs` containing the various ξs;
* `TruncatedModel` containing the two previous structures, as well as some other elements (characteristic lengths of the truncation and the indices of non-zero truncated histories).

In [5]:
struct TruncatedAllocation{I<:Integer,T<:Real}
    S_h::Vector{T}       # size of truncated histories
    Π_h::SparseMatrixCSC{T,I}# transition matrix for histories
    y0_h::Vector{T}      # current productivity levels
    a_beg_h::Vector{T}   # beginning-of-period wealth per history and per capita
    a_end_h::Vector{T}   # end-of-period wealth per history and per capita
    c_h::Vector{T}       # consumption per history and per capita
    l_h::Vector{T}       # labor supply per history and per capita
    ly_τ_h::Vector{T}    # (l y)^τ
    u_h::Vector{T}       # utility of consumption per history and per capita
    u′_h::Vector{T}      # marginal utility of consumption per history and per capita
    u′′_h::Vector{T}     # second-order derivative of utility of consumption per history and per capita
    v_h::Vector{T}       # utility of labor per history and per capita
    v′_h::Vector{T}      # marginal utility of labor per history and per capita
    resid_E_h::Vector{T} # Euler Lagrange multiplier at the history level
    nb_cc_h::I           # nb of credit constrained histories
    ind_cc_h::Vector{I}  # indices of credit constrained histories
end

struct ξs_struct{T<:Real}
    ξu0::Vector{T}
    ξu1::Vector{T}
    ξu2::Vector{T}
    ξuE::Vector{T}
    ξy ::Vector{T}
    ξv0::Vector{T}
    ξv1::Vector{T}
end  

struct TruncatedModel{I<:Integer,T<:Real}
    N::I
    refiNs::Vector{I}
    Ntot::I
    ind_h::Vector{I}
    truncatedAllocation::TruncatedAllocation{I,T}
    ξs::ξs_struct{T}
end

## Constructors

Basic constructors allowing for kwarg definition.

In [6]:
function TruncatedAllocation(;S_h::Vector{T}, Π_h::SparseMatrixCSC{T,I}, y0_h::Vector{T}, 
        a_beg_h::Vector{T}, a_end_h::Vector{T}, c_h::Vector{T}, l_h::Vector{T}, ly_τ_h::Vector{T}, 
        u_h::Vector{T}, u′_h::Vector{T}, u′′_h::Vector{T}, v_h::Vector{T}, v′_h::Vector{T}, 
        resid_E_h::Vector{T}, nb_cc_h::I, ind_cc_h::Vector{I}) where{I<:Integer,T<:Real}
    TruncatedAllocation{I,T}(S_h,Π_h,y0_h,a_beg_h,a_end_h,c_h,l_h,ly_τ_h,u_h,u′_h,u′′_h,
            v_h,v′_h,resid_E_h, nb_cc_h,ind_cc_h)    
end

function ξs_struct(;ξu0::Vector{T},ξu1::Vector{T},ξu2::Vector{T},ξuE::Vector{T},
            ξy::Vector{T},ξv0::Vector{T},ξv1::Vector{T}) where{T<:Real}
    ξs_struct{T}(ξu0,ξu1,ξu2,ξuE,ξy,ξv0,ξv1)
end  

function TruncatedModel(;N::I,refiNs::Vector{I},Ntot::I,ind_h::Vector{I},
        truncatedAllocation::TruncatedAllocation{I,T},ξs::ξs_struct{T}) where {I<:Integer,T<:Real}
    TruncatedModel{I,T}(N,refiNs,Ntot,ind_h,truncatedAllocation,ξs)
end

# Ramsey solution

## Structures

We consider three structures for the Ramsey solution:

* `Weights` containing the Pareto weights of the social welfare function. The structure contains the weights per productivity level $y$:  $(\omega_y)_y$,  represented by `ω` and $(\overline{\omega}_{y})_y = ( S_y \omega_{y})_y$,  represented by `ωb`. It also constains the weights per history $h$: $(\omega_h)_h$ (`ω_h`) and $(\overline{\omega}_{h})_h = ( S_h \omega_{h})_h$ (represented by `ωb_h`).
* `LagrangeMult` containing the Lagrange multipliers of the Ramsey program and some of their transforms. The `b`subscript refers to the history wise variable. For instance, the Lagrange multiplier on the Euler equation  $(\lambda_{c,h})_h$ is represented by the vector `λc`, $(\overline{\lambda}_{c,h})_h = ( S_h \lambda_{c,h})_h$ is represented  by `λcb`, and $(\tilde{\lambda}_{c,h})_h = (\sum_{\tilde h} \Pi_{{\tilde h} h} \lambda_{c,\tilde h})_h$ is represented  by `λct`. Same for the Lagrange multiplier on the labor FOC $(\lambda_{l,h})$ represented by `λl`, plus `λlb` and `λlt`. For the marginal value of liquidity $(\psi_{h})_y$ represented by the vector `ψ`, only add $(\overline{\psi}_{h})_h$ represented by  `ψb`. For the Lagrange mutiplier $\mu$ on the government budget constraint, it is represented by the scalar `μ` (but no transform).
* `Ramsey` containing the two previous structures, as well as the truncated model `truncatedModel`.

In [7]:
struct Weights{T<:Real}
    ω       ::Vector{T} # weights per productivity (vector of length ny)
    ωb      ::Vector{T} #     ω × S_h 
    ω_h     ::Vector{T} # weights per history (vector of length Ntot)
    ωb_h    ::Vector{T} #     ω_h × S_h 
    ω_param ::Vector{T} # parametric weights per productivity (vector of length ny)
    ωb_param::Vector{T} #     ω_param × S_h
end 
struct LagrangeMult{T<:Real}
    λc ::Vector{T} # on Euler equation 
    λct::Vector{T} #     λc lagged
    λcb::Vector{T} #     λc × S_h
    # λl ::Vector{T} # on first-order conditions for individual labor supply
    # λlt::Vector{T} #     λl lagged
    # λlb::Vector{T} #     λl × S_h
    μ  ::T         # on govt budget constraint
    Λ  ::T         # on relation inflations
    γw ::T         # on wage Phillips curve
    υ ::T          # on interest rate constraint (nominal deflated and real)
    Γ ::T          # on no profit condition 
    ψ  ::Vector{T} # social value of liquidity
    ψb ::Vector{T} #     ψ × S_h
end 
struct Ramsey{I<:Integer,T<:Real}
    # weights       ::Weights{T}
    lagrangeMult  ::LagrangeMult{T}
    truncatedModel::TruncatedModel{I,T}
end

## Constructor 

As before, we define basic constructors allowing for kwarg definition.

In [8]:
function Weights(;ω::Vector{T},ωb::Vector{T},ω_h::Vector{T},ωb_h::Vector{T},
    ω_param::Vector{T},ωb_param::Vector{T}) where{T<:Real}
#@assert size(ω) == size(ωb)
Weights(ω,ωb,ω_h,ωb_h,ω_param,ωb_param)
end 
function LagrangeMult(;λc::Vector{T},λct::Vector{T},λcb::Vector{T},
        #  λl::Vector{T},λlt::Vector{T},λlb::Vector{T},
         μ::T,Λ::T,γw::T,υ::T,Γ::T,
         ψ::Vector{T},ψb::Vector{T}) where{T<:Real}
LagrangeMult(λc,λct,λcb,μ,Λ,γw,υ,Γ,ψ,ψb)
end 
function Ramsey(;lagrangeMult::LagrangeMult{T},
   truncatedModel::TruncatedModel{I,T}) where{I<:Integer,T<:Real}
Ramsey(lagrangeMult,truncatedModel)
end